In [1]:
using Gync, Vega

In [2]:
iters = 20000
rho = 0.1
sigmay0 = 3
sigmap  = 2

config = ModelConfig(Subject(:lausanne, 1), rho, sigmay0, sigmap)

sim = runsim(config, 10_000, relprop=0.1);

MCMC Simulation of 10000 Iterations x 1 Chain...

Chain 1:   0% [1:18:11 of 1:18:16 remaining]
Chain 1:  10% [0:07:02 of 0:07:48 remaining]
Chain 1:  20% [0:06:00 of 0:07:30 remaining]
Chain 1:  30% [0:05:18 of 0:07:35 remaining]
Chain 1:  40% [0:04:38 of 0:07:43 remaining]
Chain 1:  50% [0:03:55 of 0:07:49 remaining]
Chain 1:  60% [0:03:10 of 0:07:54 remaining]
Chain 1:  70% [0:02:23 of 0:07:58 remaining]
Chain 1:  80% [0:01:36 of 0:08:00 remaining]
Chain 1:  90% [0:00:48 of 0:08:02 remaining]
Chain 1: 100% [0:00:00 of 0:08:04 remaining]



In [7]:
using Vega
import KernelDensity.kde

function plot_species(c::Mamba.AbstractChains, kde=false)
    @assert size(c,[2,3]) == (1,1)
    
    data = c.value|>vec
    
    v = kde ? plot_kde(data) : histogram(x=data, nbins = 40)
    
    title!(v, title=c.names[1])
end

function plot_kde(data::Vector)
    npoints = 2^14
    bw = maximum(data) / npoints * 5
    k  = kde(data, npoints=npoints, bandwidth=bw)
    
    v=lineplot(x=k.x,y=k.density)
    v.scales[1].zero=false
    v
end

function slices(a,dim)
    inds = Any[(:) for i in 1:length(size(a))]
    map(1:size(a,dim)) do i
        inds[dim] = i
        getindex(a,inds...)
    end
end

function plot(c::Mamba.AbstractChains)
    reduce(layer,map(plot_species,slices(c,2)))
end

@time v = plot_species(sim[:,8,1])

0.020775 seconds (25.06 k allocations: 961.409 KB)


In [8]:
# snippet for easy field-value filtering

import Vega: AbstractVegaType
import Base: getindex

function getindex(c::Vector{VegaScale}, kv::Pair)
    k,v = kv
    i=find(x->getfield(x, k) == v, c)
    c[i]
end

v.scales[:name=>"x"]

1-element Array{Vega.VegaScale,1}:
 Vega.VegaScale("x","ordinal",Vega.VegaDataRef("table_ucbhx","x",nothing,true),nothing,nothing,"width",nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing)